In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import random
import os
import cv2
import shutil
import tqdm
import glob


In [2]:
#날씨
import requests
import json

city = "Seoul" #도시
apiKey = "25ae92023210f98156f2d65b5cdb15b6"
lang = 'kr' #언어
units = 'metric' #화씨 온도를 섭씨 온도로 변경
api = f"https://api.openweathermap.org/data/2.5/weather?q={city}&APPID={apiKey}&lang={lang}&units={units}"

result = requests.get(api)
result = json.loads(result.text)

print(result['main']['temp'])
current_weather = int(result['main']['temp'])
print(current_weather)

30.89
30


In [3]:
import torch
from ultralytics import YOLO,ASSETS

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 모델 불러오기
path = "C:/Users/ham42/Downloads/Noid/Noid_projcet/best.pt"
model = YOLO(path)
model = model.to(device)



In [4]:
import gradio as gr
import PIL.Image as Image

#현재 날씨로 적정 옷 분류
if current_weather <= 15:
  iswarm = False
elif current_weather > 15:
  iswarm = True





def predict_image(img):
    """Predicts objects in an image using a YOLOv8 model with adjustable confidence and IOU thresholds."""
    results = model.predict(source=img, conf=0.4,show_labels=True,imgsz=640
    )

    for r in results:
        im_array = r.plot()
        im = Image.fromarray(im_array[..., ::-1])
        for box in r.boxes:
          class_id = int(box.data[0][-1])
          if 'jacket' in model.names[class_id]:
              top_wear = True

          if 'pants' in model.names[class_id]:
              bottom_wear = True

          if 'shirt' in model.names[class_id]:
              if 'jacket' in model.names[class_id]:
                top_wear = True

              top_wear = False

          if 'shorts' in model.names[class_id]:
              bottom_wear = False
          if 'skirt' in model.names[class_id]:
              bottom_wear = False
        
        if iswarm == top_wear:
          if iswarm == True:
            res =  ("날씨가 덥기 때문에 상의 패션은 적절하지 않습니다. 다시 선택해주세요!")
          if iswarm == False:
            res = ("날씨가 춥기 때문에 상의 패션은 적절하지 않습니다. 다시 선택해주세요!")

        if iswarm != top_wear:
            res =  ("오늘에 날씨에 딱맞게 선택하셨습니다. 상의 옷은 입으셔도 됩니다!")
        if iswarm == bottom_wear:
          if iswarm == True:
            res1 = ("날씨가 덥기 때문에 하의 패션은 적절하지 않습니다. 다시 선택해주세요!")
          elif iswarm == False:
            res1 = ("날씨가 춥기 때문에 하의 패션은 적절하지 않습니다. 다시 선택해주세요!")

        if iswarm != bottom_wear:
            res1 = ("오늘에 날씨에 딱맞게 선택하셨습니다. 하의 옷은 입으셔도 됩니다!")




    return im, res, res1


iface = gr.Interface(
    fn=predict_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
    ],
    outputs=[
       gr.Image(type="pil", label="결과"),
       gr.Textbox(label="코멘트"),
       gr.Textbox(label="코멘트")
       ],
    title="Weather Clothes Cecommendation Ai",
    description="NOID",
    examples=[
        [ASSETS / "bus.jpg", 0.25, 0.45],
        [ASSETS / "zidane.jpg", 0.25, 0.45],
    ],

)

if __name__ == "__main__":
    iface.launch()


c:\Users\ham42\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.



0: 640x416 1 jacket, 1 shirt, 1 pants, 1 shoe, 594.2ms
Speed: 0.0ms preprocess, 594.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 416)

0: 640x544 2 jackets, 1 pants, 1 skirt, 1 dress, 781.1ms
Speed: 112.7ms preprocess, 781.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 544)
